In [ ]:
from elasticsearch import Elasticsearch

# Connect to Elasticsearch
es = Elasticsearch("http://localhost:9200")

# Define the aggregation query
query = {
    "size": 0,
    "aggs": {
        "type_counts": {
            "terms": {
                "field": "type.keyword",  # Use .keyword to aggregate on text fields
                "size": 100  # Adjust size to return more unique values if needed
            }
        }
    }
}

# Execute the search query
index_name = "your_index_name"  # Replace with your actual index name
response = es.search(index=index_name, body=query)

# Extract and print the aggregation results
buckets = response["aggregations"]["type_counts"]["buckets"]
for bucket in buckets:
    print(f'Type: {bucket["key"]}, Count: {bucket["doc_count"]}')

In [1]:
url = "https://data.wellcomecollection.org/catalogue/v2/works.json.gz"

from sources.catalogue.concepts_source import CatalogueConceptsSource
from transformers.catalogue.raw_concept import RawCatalogueConcept

source = CatalogueConceptsSource(url)

counter = 0
all_concepts = []

for entity in source.stream_raw():
    concept = RawCatalogueConcept(entity, {})
    
    if concept.is_concept:
        
        counter += 1
        all_concepts.append((concept.source_concept_id, concept.wellcome_id, concept.source, concept.type, concept.label))
        
        if counter % 10000 == 0:
            print(counter)

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000
1290000
1300000
1310000
1320000
1330000
1340000
1350000
1360000
1370000
1380000
1390

In [2]:
deduplicated_concepts = []
ids = set()
for concept in all_concepts:
    
    if concept[0] + concept[1] not in ids:
        deduplicated_concepts.append(concept)
        ids.add(concept[0] + concept[1])

In [3]:
from collections import defaultdict

source_id_grouping = defaultdict(list)
for concept in deduplicated_concepts:
    source_id_grouping[concept[0]].append(concept)

In [4]:
oops = []
for key, value in source_id_grouping.items():
    if len(value) > 1:
        oops.append(value)



In [147]:
len(oops)

3854

In [6]:
counts = defaultdict(int)
for value in oops:
    if len(value) == 5:
        print(value)
    counts[len(value)] += 1

print(counts)

[('united states', 'mkhspjcb', 'label-derived', 'Place', 'United States'), ('united states', 'h4yzuzfr', 'label-derived', 'Person', 'United States'), ('united states', 'gzp2c8kw', 'label-derived', 'Genre', 'United States'), ('united states', 'mp427cuy', 'label-derived', 'Organisation', 'United States'), ('united states', 'rqnzj8mg', 'label-derived', 'Period', 'United States')]
defaultdict(<class 'int'>, {2: 3658, 4: 8, 3: 187, 5: 1})


In [173]:
counts = defaultdict(int)

for value in oops:
    
    types = tuple(sorted([v[3] for v in value]))
    
    if set(v[2] for v in value) == {'label-derived'} and types not in [('Agent', 'Person'), ('Agent', 'Organisation'), ('Concept', 'Genre'), ('Agent', 'Concept'), ('Concept', 'Person'), ('Concept', 'Organisation'), ('Concept', 'Period'), ('Concept', 'Place'), ('Agent', 'Concept', 'Person')]:
        counts[types] += 1
        print(value[0][0], types)

# for key, value in sorted(counts.items(), key=lambda i: -i[1]):
#     print(key, value)

church of england ('Organisation', 'Person')
ancient ('Concept', 'Period', 'Person', 'Place')
great britain ('Organisation', 'Person', 'Place')
london (england) ('Organisation', 'Person', 'Place')
ireland ('Person', 'Place')
new york (state) ('Person', 'Place')
byzantine empire ('Organisation', 'Place')
royal veterinary college ('Agent', 'Concept', 'Organisation')
united states. surgeon-general's office ('Organisation', 'Person')
jamaica ('Person', 'Place')
dublin (ireland) ('Person', 'Place')
united states ('Genre', 'Organisation', 'Period', 'Person', 'Place')
norwich (england) ('Person', 'Place')
france ('Concept', 'Organisation', 'Person', 'Place')
hong kong ('Organisation', 'Place')
pennsylvania ('Organisation', 'Person', 'Place')
portugal ('Person', 'Place')
paris (france) ('Organisation', 'Person', 'Place')
connecticut ('Organisation', 'Person', 'Place')
university of edinburgh ('Agent', 'Concept', 'Organisation', 'Person')
wales ('Person', 'Place')
history ('Concept', 'Genre', '

In [151]:
counts = defaultdict(int)

for value in oops:
    types = tuple(sorted([v[3] for v in value]))
    counts[types] += 1
    
for key, value in sorted(counts.items(), key=lambda i: -i[1]):
    print(key, value)
    

('Agent', 'Person') 2210
('Concept', 'Genre') 300
('Concept', 'Person') 253
('Organisation', 'Place') 203
('Organisation', 'Person') 192
('Agent', 'Agent', 'Person') 107
('Agent', 'Organisation') 101
('Concept', 'Organisation') 84
('Person', 'Place') 69
('Organisation', 'Organisation') 51
('Concept', 'Place') 47
('Agent', 'Concept') 45
('Person', 'Person') 36
('Agent', 'Concept', 'Person') 29
('Agent', 'Agent') 27
('Organisation', 'Person', 'Place') 19
('Concept', 'Concept') 18
('Concept', 'Organisation', 'Person') 13
('Concept', 'Period') 10
('Meeting', 'Meeting') 5
('Agent', 'Concept', 'Organisation') 4
('Agent', 'Person', 'Person') 4
('Concept', 'Organisation', 'Person', 'Place') 3
('Agent', 'Concept', 'Organisation', 'Person') 3
('Concept', 'Person', 'Place') 3
('Concept', 'Genre', 'Person') 2
('Meeting', 'Organisation') 2
('Concept', 'Organisation', 'Place') 2
('Genre', 'Place') 2
('Concept', 'Period', 'Person', 'Place') 1
('Organisation', 'Place', 'Place') 1
('Genre', 'Organisati

In [66]:
for value in oops:
    sources = tuple(sorted([v[2] for v in value]))
    types = tuple(sorted([v[3] for v in value]))

    if sources != ("lc-names", "nlm-mesh") and sources != ("fihrist", "viaf"):
        if sources != ("lc-names", "lc-names") and len(sources) == 2:
            
            print(sources, types, value)

('nlm-mesh', 'nlm-mesh') ('Concept', 'Concept') [('D010604', 'mgg8epvw', 'nlm-mesh', 'Concept'), ('D010604', 'mn7m7h3a', 'nlm-mesh', 'Concept')]
('nlm-mesh', 'nlm-mesh') ('Concept', 'Concept') [('D006778', 'kn5thm26', 'nlm-mesh', 'Concept'), ('D006778', 'xjx9rnqh', 'nlm-mesh', 'Concept')]
('lc-subjects', 'nlm-mesh') ('Concept', 'Concept') [('sh85023418', 'j63yjcwj', 'lc-subjects', 'Concept'), ('sh85023418', 'd2v7fqx6', 'nlm-mesh', 'Concept')]
('nlm-mesh', 'nlm-mesh') ('Concept', 'Concept') [('D013405', 'pwcxjg8r', 'nlm-mesh', 'Concept'), ('D013405', 'hverncc5', 'nlm-mesh', 'Concept')]
('nlm-mesh', 'nlm-mesh') ('Concept', 'Concept') [('D008880', 'xpbmqqq6', 'nlm-mesh', 'Concept'), ('D008880', 'z9rm867e', 'nlm-mesh', 'Concept')]
('nlm-mesh', 'nlm-mesh') ('Concept', 'Concept') [('D001523', 'd95jwhsd', 'nlm-mesh', 'Concept'), ('D001523', 'cfryf4w8', 'nlm-mesh', 'Concept')]
('nlm-mesh', 'nlm-mesh') ('Concept', 'Concept') [('D020521', 'xsvh8ssy', 'nlm-mesh', 'Concept'), ('D020521', 'hmqyhr72

In [75]:

for value in oops:
    types = tuple(sorted([v[3] for v in value]))
    
    if types == ("Meeting", "Meeting"):
        # Example: [('n2001128565', 'j7bxqtsb', 'lc-names', 'Meeting'), ('n2001128565', 'dwupgks2', 'nlm-mesh', 'Meeting')]
        # Incorrect source!
        # We can ignore these -- they should be resolved at source
        pass
    if types == ("Agent", "Agent"):
        # Same as above
        pass
    if types == ('Agent', 'Person'):
        # [('n82024440', 'w3d7f23y', 'lc-names', 'Organisation'), ('n82024440', 'z4sqb3wh', 'nlm-mesh', 'Organisation')]
        # Same as above
        print(value)
    if types == ('Concept', 'Genre'):
        #print(value)
        pass

[('hood, basil, 1876-1978', 'ushvgaw9', 'label-derived', 'Person'), ('hood, basil, 1876-1978', 'ta9u9k7t', 'label-derived', 'Agent')]
[('n50015907', 'kxckf4kv', 'lc-names', 'Person'), ('n50015907', 't3zgq34k', 'lc-names', 'Agent')]
[('silvester, thomas hookham', 'fpnc3f7k', 'label-derived', 'Agent'), ('silvester, thomas hookham', 'xj9pemqt', 'label-derived', 'Person')]
[('n79060434', 'd63pahub', 'lc-names', 'Person'), ('n79060434', 'xv4efebk', 'lc-names', 'Agent')]
[('n86805727', 'tjvvwu8j', 'lc-names', 'Person'), ('n86805727', 'wp4x99b6', 'nlm-mesh', 'Agent')]
[('n50001534', 'ut6hw3r3', 'lc-names', 'Person'), ('n50001534', 'cvmakqqd', 'lc-names', 'Agent')]
[('n84805280', 'j96rp92d', 'lc-names', 'Person'), ('n84805280', 'zjjzm7k8', 'lc-names', 'Agent')]
[('n79003254', 'th8r4uq3', 'lc-names', 'Person'), ('n79003254', 'q9tzvm5d', 'lc-names', 'Agent')]
[('n85112114', 'axgzt3xr', 'lc-names', 'Person'), ('n85112114', 'zz98s6vd', 'lc-names', 'Agent')]
[('n83121239', 'jjebzwdn', 'lc-names', '

In [ ]:
# ('Agent', 'Person') 1745
# ('Agent', 'Agent', 'Person') 107
# ('Agent', 'Person', 'Person') 4

# 1800+ are for 'Agent' and 'Person'

# ('Organisation', 'Place') 165
# ('Organisation', 'Place', 'Place') 1
# ('Organisation', 'Person', 'Place') 5

# Another 170+ are for 'Organisation' and 'Place'

# ('Organisation', 'Organisation') 51 => Data quality issue

# ('Concept', 'Place') 3
# ('Organisation', 'Person') 6
# ('Concept', 'Person') 5
# ('Concept', 'Genre') 2

# ('Concept', 'Concept') 18
# ('Person', 'Person') 36 => Data quality issue (also viaf and fihrist)
# ('Agent', 'Agent') 27 => Data quality issue
# ('Meeting', 'Meeting') 5 => Data quality issue

Some duplicate MeSH IDs due to qualifier issue:

# D013405Q000517 vs D013405


[('D013405', 'pwcxjg8r', 'nlm-mesh', 'Concept'), ('D013405', 'hverncc5', 'nlm-mesh', 'Concept')]
[('D008880', 'xpbmqqq6', 'nlm-mesh', 'Concept'), ('D008880', 'z9rm867e', 'nlm-mesh', 'Concept')]
[('D001523', 'd95jwhsd', 'nlm-mesh', 'Concept'), ('D001523', 'cfryf4w8', 'nlm-mesh', 'Concept')]
[('D020521', 'xsvh8ssy', 'nlm-mesh', 'Concept'), ('D020521', 'hmqyhr72', 'nlm-mesh', 'Concept')]

In [75]:
import re


counter = 0
for concept in deduplicated_concepts:
    if concept[3] == "Agent":
            print(concept)

            counter += 1

print(counter)


('worthington, robert, (fl.1831)', 'qdhcpgja', 'label-derived', 'Agent', 'Worthington, Robert, (fl.1831)')
('silvester, thomas hookham', 'fpnc3f7k', 'label-derived', 'Agent', 'Silvester, Thomas Hookham.')
('spencer, michael', 'taxaqg9m', 'label-derived', 'Agent', 'Spencer, Michael')
('fuller, watson, 1935-', 'k735ab8h', 'label-derived', 'Agent', 'Fuller, Watson, 1935-')
('butter, jean', 'p6wr9wbr', 'label-derived', 'Agent', 'Butter, Jean')
('caroline gunn', 'yb4qd2nz', 'label-derived', 'Agent', 'Caroline Gunn')
('robert harris', 'b4x8mzbm', 'label-derived', 'Agent', 'Robert Harris')
('n80119944', 'z43znvbw', 'lc-names', 'Agent', 'Hippocrates. Aphorisms. Italian')
('n2004015514', 'truka8rs', 'lc-names', 'Agent', 'Nicaise, E. (Edouard), 1838-1896. Chirurgie de maître Henri de Mondeville.')
('heather spears', 'n282kp48', 'label-derived', 'Agent', 'Heather Spears')
('david livingstone', 'q79w8ptc', 'label-derived', 'Agent', 'David Livingstone')
('t. hodgetts', 'rabzpar2', 'label-derived', 

In [8]:
import re

counter = 0
mesh_mappings = {}
for concept in deduplicated_concepts:
    if concept[2] == "nlm-mesh" and concept[0][0] != "D":
        mesh_mappings[concept[1]] = concept

        counter += 1

print(counter)


614


In [27]:
import csv
import json

full_info = defaultdict(list)
with open("/Users/brychtas/Downloads/works.json-3", "r") as f:
    for i, work in enumerate(f):
        work = json.loads(work)
        for concept_key in ["subjects", "genres", "contributors"]:
            for item in work.get(concept_key, []):
                if "concepts" in item:
                    concept_id = item["concepts"][0].get("id")
                else:
                    concept_id = item["agent"]["id"]

                if concept_id and concept_id in mesh_mappings:
                    full_info[concept_id].append(work)

print(len(mesh_mappings))
print(len(full_info))

with open('incorrect_ontologies.csv', 'w') as f:
    writer = csv.DictWriter(f, fieldnames=["Concept source id", "Concept Wellcome id", "Concept label", "Concept source ontology", "Works referencing this concept", "Possible solution"])
    writer.writeheader()

    for key, mapping in mesh_mappings.items():
        work_ids = {item["id"] for item in full_info[key]}
        
        if mapping[0][0] == "n":
            solution = "This concept is classified under MeSH but is most likely a a Library of Congress Name."
        elif mapping[0][:2] == "sh":
            solution = "This concept is classified under MeSH but is most likely a a Library of Congress Subject Heading."
        else:
            solution = "The source id of this concept is not valid."
        
        writer.writerow({
            "Concept source id": mapping[0],
            "Concept Wellcome id": mapping[1],
            "Concept source ontology": mapping[2],
            "Concept label": mapping[-1],
            "Works referencing this concept": ", ".join([f"https://wellcomecollection.org/works/{i}" for i in work_ids]),
            "Possible solution": solution
        })



In [28]:
full_info["s4ajuxpw"][0]["identifiers"]

[{'identifierType': {'id': 'sierra-system-number',
   'label': 'Sierra system number',
   'type': 'IdentifierType'},
  'value': 'b16557815',
  'type': 'Identifier'},
 {'identifierType': {'id': 'sierra-identifier',
   'label': 'Sierra identifier',
   'type': 'IdentifierType'},
  'value': '1655781',
  'type': 'Identifier'}]

In [15]:
import json
oops_ids = set()
for items in oops:
    for item in items:
        oops_ids.add(item[1])

work_lookup = defaultdict(list)
works = {}
with open("/Users/brychtas/Downloads/works.json-3", "r") as f:
    for i, work in enumerate(f):
        work = json.loads(work)
        for concept_key in ["subjects", "genres", "contributors"]:
            for item in work.get(concept_key, []):
                if "concepts" in item:
                    concept_id = item["concepts"][0].get("id")
                else:
                    concept_id = item["agent"]["id"]
                
                if concept_id and concept_id in oops_ids:
                    work_id = work["id"]
                    work_lookup[concept_id].append(work_id)
                    works[work_id] = work


In [71]:
for items in oops:
    types = tuple(sorted([item[3] for item in items]))
    if types == ("Agent", "Person"):
            for item in items:
                if item[3] == "Agent":
                    sources = set()
                    
                    for work_id in work_lookup[item[1]]:
                        #if works[work_id]["identifiers"][0]["identifierType"]["label"] == "Calm RecordIdentifier":
                            #print(item[-1])
                            source = works[work_id]["identifiers"][0]["identifierType"]["label"]
                            if 'miro' in source.lower():
                                print(source, works[work_id]["identifiers"][0], item)
                            sources.add(source)
                        #print(works[work_id]["identifiers"], works[work_id]["contributors"], item[-2])
                    
                    
                    #print(len(work_lookup[item[1]]), ", ".join(sources), item[-2], end="; ")
            # print("\n")

Miro image number {'identifierType': {'id': 'miro-image-number', 'label': 'Miro image number', 'type': 'IdentifierType'}, 'value': 'L0017586', 'type': 'Identifier'} ('campbell', 's86vh6bb', 'label-derived', 'Agent', 'Campbell')
Miro image number {'identifierType': {'id': 'miro-image-number', 'label': 'Miro image number', 'type': 'IdentifierType'}, 'value': 'L0072280', 'type': 'Identifier'} ('thurnam, john', 'aszrc7gm', 'label-derived', 'Agent', 'Thurnam, John')
Miro image number {'identifierType': {'id': 'miro-image-number', 'label': 'Miro image number', 'type': 'IdentifierType'}, 'value': 'L0072279', 'type': 'Identifier'} ('thurnam, john', 'aszrc7gm', 'label-derived', 'Agent', 'Thurnam, John')
Miro image number {'identifierType': {'id': 'miro-image-number', 'label': 'Miro image number', 'type': 'IdentifierType'}, 'value': 'L0004888', 'type': 'Identifier'} ('lafosse', 'mvaf55ah', 'label-derived', 'Agent', 'Lafosse')
Miro image number {'identifierType': {'id': 'miro-image-number', 'labe

In [26]:
print(len(oops_ids), len(oops))

7914 3854


In [28]:
for items in oops:
    for item in items:
        sources = set()
        for work_id in work_lookup[item[1]]:
            print(works[work_id])
            sources.add(works[work_id]["identifiers"][0]["identifierType"]["label"])
                    
    break

{'succeededBy': [{'totalDescendentParts': 0, 'id': 'vq9xyvfp', 'totalParts': 0, 'title': "St Mary's Hospital, Plaistow: Sir Robert Archibald. Photograph, 1904.", 'type': 'Work'}, {'totalDescendentParts': 0, 'id': 'zxad8qkt', 'totalParts': 0, 'title': "St Mary's Hospital, Plaistow: main ward. Photograph, 1904.", 'type': 'Work'}, {'totalDescendentParts': 0, 'id': 'yjdwypsr', 'totalParts': 0, 'title': "St Mary's Hospital, Plaistow: patients, children and a nurse. Photograph, 1904.", 'type': 'Work'}, {'totalDescendentParts': 0, 'id': 'b22r6jgn', 'totalParts': 0, 'title': "St Mary's Hospital, Plaistow: children having a meal outdoors. Photograph, 1904.", 'type': 'Work'}, {'totalDescendentParts': 0, 'id': 'u76suqk2', 'totalParts': 0, 'title': "St Mary's Hospital, Plaistow: Basil Hood and Robert Archibald with patients and nurses, having tea outdoors. Photograph, 1904.", 'type': 'Work'}, {'totalDescendentParts': 0, 'id': 'bkvs4qc2', 'totalParts': 0, 'title': "St Mary's Hospital, Plaistow: chi

In [76]:
import re

agent_ids = set()
counter = 0
for concept in deduplicated_concepts:
    if concept[3] == "Agent":
        agent_ids.add(concept[1])

work_lookup = defaultdict(list)
works = {}
with open("/Users/brychtas/Downloads/works.json-3", "r") as f:
    for i, work in enumerate(f):
        work = json.loads(work)
        for concept_key in ["subjects", "genres", "contributors"]:
            for item in work.get(concept_key, []):
                if "concepts" in item:
                    concept_id = item["concepts"][0].get("id")
                else:
                    concept_id = item["agent"]["id"]

                if concept_id and concept_id in agent_ids:
                    work_id = work["id"]
                    work_lookup[concept_id].append(work_id)
                    works[work_id] = work

In [132]:
from collections import Counter
import boto3
import json

session = boto3.Session(profile_name="platform-developer")
s3 = session.client("s3")

counter = Counter()

all_labels = set()
for concept in deduplicated_concepts:
    if concept[3] == "Agent":
            work_id = work_lookup[concept[1]][0]
            source_label = works[work_id]["identifiers"][0]["identifierType"]["label"]
            if 'Miro' in source_label:
                identifier = works[work_id]["identifiers"][0]['value']
                result = s3.list_objects_v2(Bucket="wellcomecollection-vhs-sourcedata-miro", Prefix=identifier)
                file = s3.get_object(Bucket="wellcomecollection-vhs-sourcedata-miro", Key=result["Contents"][-1]["Key"])
                content = json.loads(file['Body'].read())

                print(concept[-1], content['image_source_code'])
                
                all_labels.add(concept[-1])
            counter[source_label] += 1
print(len(all_labels))
for l in all_labels:
    print(l)

Caroline Gunn GUN
Robert Harris WEL
Heather Spears SPE
David Livingstone WEL
T. Hodgetts WEL
Campbell WEL
Michael Frank FRN
S. Cousins WEL
Royal Veterinary College RVC
Yirui Sun SUN
Carole Reeves RVS
Dave Farnham FAR
H Kuper KHA
Anne Weston, Francis Crick Institute WST
Wessex Reg. Genetics Centre WES
P Bedford/RVC RVC
Elliot & Fry WEL
Alexander von Humboldt WEL
Lillias Anna Hamilton WEL
Krsnalilasukamuni WEL
Fernan Federici & Jim Haseloff FED
NIMR, Francis Crick Institute MOH
David Gregory & Debbie Marshall GRE
Odra Noel ANA
William J Moore/Univ. Dundee BIO
Arnold & Sons WEL
Valentine Abbatt WEL
Thomas de Bry WEL
University of Edinburgh EMV
Arthur Schuster WEL
Dr David Furness FUR
Dr Steve Wilson WIL
Davis, Joseph Barnard WEL
Thurnam, John WEL
Gull Lab of courtesy of Catarina Gadelha GWD
John Thomson WEL
Matthias Medtmann WEL
Bhawani. Das WEL
M Rowland RMA
Gustave Dore WEL
Kevin Mackenzie, University of Aberdeen MAK
James King-Holmes JKH
Chris Guerin CGU
Dr. Xiao-nong Wang, Human Dendr

KeyboardInterrupt: 

In [114]:
work_lookup["baebkkmd"][0]
works["umnpt5zg"]

{'succeededBy': [],
 'production': [{'label': '1964-1968',
   'agents': [],
   'dates': [{'label': '1964-1968', 'type': 'Period'}],
   'type': 'ProductionEvent',
   'places': []}],
 'physicalDescription': '1 box 1 folder',
 'subjects': [{'label': 'Neurology',
   'concepts': [{'id': 'rfjvsmd4',
     'identifiers': [{'identifierType': {'id': 'label-derived',
        'label': 'Identifier derived from the label of the referent',
        'type': 'IdentifierType'},
       'value': 'neurology',
       'type': 'Identifier'}],
     'label': 'Neurology',
     'type': 'Concept'}],
   'identifiers': [{'identifierType': {'id': 'label-derived',
      'label': 'Identifier derived from the label of the referent',
      'type': 'IdentifierType'},
     'value': 'neurology',
     'type': 'Identifier'}],
   'id': 'rfjvsmd4',
   'type': 'Subject'},
  {'label': 'Migraine',
   'concepts': [{'id': 'zwnvyfaa',
     'identifiers': [{'identifierType': {'id': 'label-derived',
        'label': 'Identifier derived 

In [133]:
len(agent_ids)

10817

In [146]:
from collections import Counter
import boto3
import json

session = boto3.Session(profile_name="platform-developer")
s3 = session.client("s3")

counter = Counter()

all_labels = set()
for concept in deduplicated_concepts:
    if concept[3] == "Agent":
        work_id = work_lookup[concept[1]][0]
        source_label = works[work_id]["identifiers"][0]["identifierType"]["label"]
        if 'Sierra' in source_label:

            if concept[2] == "label-derived":
                all_labels.add((concept[-1], concept[1]))
        counter[source_label] += 1


for label in all_labels:
    print(label)
    
len(all_labels)

('Akattiyar. Paripūraṇam 1200.', 'zt8xvu9r')
('Karuvūrār. Civañāṉa pōtam 43.', 'p5k96nsd')
('Rubenii, A. De re vestiaria.', 'rbhupn6j')
('Moseley, Dr. Review of the Report of the London College of Physicians on vaccination.', 'j5ucypcy')
('Tortora, Luigi. Considerazioni fisico-cerusiche.', 'fn3xfgwv')
('Cattier, Isaac, active 1637-1653. Observationes medicinales rarae.', 'xyp2axb4')
('Neander, Johann, approximately 1596-approximately 1630. Tabacologia.', 'rjnt86q9')
('Wood, Andrew, 1810-1881. Time is drawing near.', 'pfmrza8k')
('Ānandapūrṇa Vidyāsāgara, active 13th century. Khaṇḍanaphakkikāvibhajana.', 'qytufqs9')
('Saalmann, Ferdinand, 1732-1792. Descriptio pleuritidis, peripneumoniae, pleuropneumoniae et anginae earumque curatio.', 'pfsrywbz')
('Chapman, Edward Seymour. Post-mortem appearances in a case of enteric pyelitis.', 'yrar8kmw')
('Thorius, Raphael, -1625. Hymnus tabaci.', 'cp5bdp9y')
('Astari, Biagio. De curandis febribus.', 'st5n3d8g')
('Goodall, Mr. Pneumatic engine, for 

902